In [5]:
#attempted transshipment
using JuMP
import DelimitedFiles
import HiGHS

open(joinpath(@__DIR__, "transp.txt"), "w") do io
    print(
        io,
        """
             . 広瀬中   仙台青    蒲町小   長町中  七郷小   新田小    仙台高  南光台    錦ケ丘  上杉山   宮城野   仙台商   岩切小   榴岡小   東長町   富沢小   富沢中  DSSK      DSMC    DSJC  EVACUEES
          TTWR 10.71	5.25	5.23	2.85	6.46	5.29	4.80	5.31	10.79	1.65	2.31	7.46	7.68	1.47	4.33	5.32	4.38	2.27	8.91	10.57  1178
          ZUHO 9.99 	5.15	5.96	2.30	7.21	6.35	4.58	6.27	9.92	2.36	3.37	8.10	8.76	2.54	4.12	4.54	3.57	2.26	8.24	10.90  60
          OSAK 7.65 	2.05	8.60	5.35	9.81	7.72	1.46	5.91	8.07	2.65	5.24	6.18	9.36	4.45	7.26	7.13	6.21	1.20	5.80	8.15   100000
          TDAI 10.57	5.32	5.34	2.54	6.59	5.60	4.82	5.66	10.59	1.93	2.61	7.75	8.03	1.78	4.11	4.98	4.03	2.33	8.79	10.78  5000
          AERT 11.05	5.20	5.04	3.56	6.24	4.64	4.84	4.54	11.26	1.20	1.77	6.85	6.94	0.93	4.86	6.07	5.13	2.32	9.22	10.15  773
          TEPC 10.77	4.81	5.43	3.86	6.61	4.78	4.47	4.33	11.03	0.78	2.04	6.49	6.95	1.25	5.24	6.33	5.39	1.98	8.93	9.74   12500
          SIPO 10.21	5.89	13.99	12.75	14.91	11.12	6.61	7.37	11.83	8.49	10.52	3.68	10.65	10.19	14.46	14.79	13.85	8.18	8.97	0.62   1653
          MEDI 9.60 	3.88	6.50	3.89	7.71	5.92	3.45	4.99	9.85	0.99	3.21	6.51	7.95	2.39	5.59	6.14	5.17	0.90	7.77	9.34   346
          SSTA 11.20	5.44	4.83	3.35	6.04	4.60	5.06	4.71	11.37	1.47	1.66	7.10	6.99	0.81	4.61	5.88	4.96	2.53	9.39	10.41  75608
          AQUA 20.33	13.61	6.51	11.64	5.73	4.97	13.62	7.72	20.85	10.09	7.88	10.96	4.19	8.75	11.16	13.84	13.25	11.60	18.47	15.06  913
          MOPS 21.04	14.37	6.77	12.06	5.86	5.67	14.36	8.57	21.52	10.78	8.48	11.84	5.09	9.35	11.45	14.18	13.63	12.28	19.18	15.93  10000
          YURT 12.14	5.55	9.58	9.79	10.35	6.41	6.02	2.70	13.27	5.35	6.47	1.03	5.92	6.43	11.09	12.22	11.26	5.91	10.43	5.28   11467
          CAST 8.98 	4.51	6.99	2.90	8.24	7.21	3.87	6.69	8.90	2.68	4.27	8.07	9.47	3.42	4.85	4.72	3.77	1.96	7.25	10.53  87
          RAKU 13.11	7.21	2.96	3.98	4.13	3.19	6.90	4.78	13.26	3.13	0.68	7.90	5.95	1.21	4.39	6.46	5.68	4.42	11.30	11.60  27000
          DSSK 8.82 	3.01	7.39	4.57	8.60	6.58	2.56	5.16	9.16	1.49	4.03	6.13	8.40	3.24	6.37	6.64	5.68	0.00	6.98	8.67   999999
          DSMC 1.85 	4.96	14.30	9.96	15.55	13.49	4.82	11.11	2.85	8.46	11.04	9.68	14.82	10.23	11.94	10.56	9.92	7.00	0.00	8.93   999999
          DSJC 10.18	6.29	14.62	13.28	15.55	11.76	7.01	8.01	11.86	9.06	11.14	4.31	11.27	10.79	15.01	15.28	14.34	8.70	9.04	0.00   999999
        VACANCIES 15232 14955 12950 16378 12195 13261 21261 10464 14474 14445 14733 24328 16014 13779 12595 15111 18578 999999 999999 999999 0
        """,
    )
    return
end

function read_data(filename::String)
    data = DelimitedFiles.readdlm(filename)
    rows, columns = data[2:end, 1], data[1, 2:end]
    return Containers.DenseAxisArray(data[2:end, 2:end], rows, columns)
end

data = read_data(joinpath(@__DIR__, "transp.txt"))
print(data)

function solve_transportation_problem(data::Containers.DenseAxisArray)
    # Get the set of supplies and demands
    O, D = axes(data)
    # Drop the EVACUEES and VACANCIES nodes from our sets
    O, D = setdiff(O, ["VACANCIES"]), setdiff(D, ["EVACUEES"])
    model = Model(HiGHS.Optimizer)
    set_silent(model)
    @variable(model, x[o in O, d in D] >= 0)
    # Remove arcs with "." cost by fixing them to 0.0.
    for o in O, d in D
        if data[o, d] == "."
            fix(x[o, d], 0.0; force = true)
        end
    end
    @objective(
        model,
        Min,
        sum(data[o, d] * x[o, d] for o in O, d in D if data[o, d] != "."),
    )
    @constraint(model, [o in O], sum(x[o, :]) == data[o, "EVACUEES"])
    @constraint(model, [d in D], sum(x[:, d]) <= data["VACANCIES", d])
    optimize!(model)
    # Pretty print the solution in the format of the input
    print("    ", join(lpad.(D, 7, ' ')))
    for o in O
        print("\n", o)
        for d in D
            if isapprox(value(x[o, d]), 0.0; atol = 1e-6)
                print("      .")
            else
                print(" ", lpad(value(x[o, d]), 6, ' '))
            end
        end
    end
    return
end

print("\nSolution:\n")
solve_transportation_problem(data)

#########################################

function calculate_numerical_derivative(data::JuMP.Containers.DenseAxisArray{Any, 2}, epsilon::Float64 = 1)
    O, D = axes(data)
    O, D = setdiff(O, ["VACANCIES"]), setdiff(D, ["EVACUEES"])
    num_rows, num_cols = length(O), length(D)
    derivative_matrix = zeros(Float64, num_rows, num_cols)

    for (i, o) in enumerate(O), (j, d) in enumerate(D)
        original_cost = data[o, d]

        if original_cost isa Number
            # Increase the cost slightly and re-solve
            data[o, d] = original_cost + epsilon
            increased_cost_model = Model(HiGHS.Optimizer)
            set_silent(increased_cost_model)
            @variable(increased_cost_model, x[ii in O, jj in D] >= 0)

            for ii in O, jj in D
                if data[ii, jj] == "."
                    fix(x[ii, jj], 0.0; force = true)
                end
            end

            @objective(
                increased_cost_model,
                Min,
                sum(data[ii, jj] * x[ii, jj] for ii in O, jj in D if data[ii, jj] isa Number),
            )

            @constraint(increased_cost_model, [ii in O], sum(x[ii, :]) == data[ii, "EVACUEES"])
            @constraint(increased_cost_model, [jj in D], sum(x[:, jj]) <= data["VACANCIES", jj])

            optimize!(increased_cost_model)
            increased_cost_objective = objective_value(increased_cost_model)

            # Decrease the cost slightly and re-solve
            data[o, d] = original_cost - epsilon
            decreased_cost_model = Model(HiGHS.Optimizer)
            set_silent(decreased_cost_model)
            @variable(decreased_cost_model, x[ii in O, jj in D] >= 0)

            for ii in O, jj in D
                if data[ii, jj] == "."
                    fix(x[ii, jj], 0.0; force = true)
                end
            end

            @objective(
                decreased_cost_model,
                Min,
                sum(data[ii, jj] * x[ii, jj] for ii in O, jj in D if data[ii, jj] isa Number),
            )

            @constraint(decreased_cost_model, [ii in O], sum(x[ii, :]) == data[ii, "EVACUEES"])
            @constraint(decreased_cost_model, [jj in D], sum(x[:, jj]) <= data["VACANCIES", jj])

            optimize!(decreased_cost_model)
            decreased_cost_objective = objective_value(decreased_cost_model)

            # Calculate the numerical derivative
            derivative = (increased_cost_objective - decreased_cost_objective) / (2 * epsilon)
            derivative_matrix[i, j] = derivative

            # Reset the original cost
            data[o, d] = original_cost
        end
    end

    return derivative_matrix
end

# Calculate and print the numerical derivative matrix
epsilon = 1.0
derivative_matrix = calculate_numerical_derivative(data, epsilon)
println("\nNumerical Derivative Matrix:")
for row in eachrow(derivative_matrix)
    println(row)
end

2-dimensional DenseAxisArray{Any,2,...} with index sets:
    Dimension 1, Any["TTWR", "ZUHO", "OSAK", "TDAI", "AERT", "TEPC", "SIPO", "MEDI", "SSTA", "AQUA", "MOPS", "YURT", "CAST", "RAKU", "DSSK", "DSMC", "DSJC", "VACANCIES"]
    Dimension 2, Any["広瀬中", "仙台青", "蒲町小", "長町中", "七郷小", "新田小", "仙台高", "南光台", "錦ケ丘", "上杉山"  …  "仙台商", "岩切小", "榴岡小", "東長町", "富沢小", "富沢中", "DSSK", "DSMC", "DSJC", "EVACUEES"]
And data, a 18×21 Matrix{Any}:
    10.71      5.25      5.23      2.85      6.46      5.29      4.8       5.31     10.79      1.65      2.31      7.46      7.68      1.47      4.33      5.32      4.38       2.27       8.91      10.57    1178
     9.99      5.15      5.96      2.3       7.21      6.35      4.58      6.27      9.92      2.36      3.37      8.1       8.76      2.54      4.12      4.54      3.57       2.26       8.24      10.9       60
     7.65      2.05      8.6       5.35      9.81      7.72      1.46      5.91      8.07      2.65      5.24      6.18      9.36      4.45      7.2

In [9]:
#Import necessary packages
import Pkg; Pkg.add("OpenStreetMapX")

import JuMP
import HiGHS
import LinearAlgebra
import Plots
import Distances
import GeoJSON
import OpenStreetMapX
import GLPK

   Resolving package versions...
   Installed OpenSSL_jll ──── v3.0.10+0
   Installed StableRNGs ───── v1.0.0
   Installed LibExpat ─────── v0.6.1
   Installed ArnoldiMethod ── v0.2.0
   Installed Inflate ──────── v0.1.3
   Installed LoggingExtras ── v1.0.2
   Installed SimpleTraits ─── v0.9.4
   Installed protoc_jll ───── v3.21.12+1
   Installed ProtoBuf ─────── v0.11.5
   Installed OpenStreetMapX ─ v0.3.4
   Installed Graphs ───────── v1.8.0
    Updating `~/Project.toml`
  [86cd37e6] + OpenStreetMapX v0.3.4
    Updating `~/Manifest.toml`
  [ec485272] + ArnoldiMethod v0.2.0
  [d1d4a3ce] + BitFlags v0.1.7
  [f0e56b4a] + ConcurrentUtilities v2.2.1
  [460bff9d] + ExceptionUnwrapping v0.1.9
  [86223c79] + Graphs v1.8.0
  [cd3eb016] + HTTP v1.9.14
  [d25df0c9] + Inflate v0.1.3
  [522f3ed2] + LibExpat v0.6.1
  [e6f89c97] + LoggingExtras v1.0.2
  [4d8831e6] + OpenSSL v1.4.1
  [86cd37e6] + OpenStreetMapX v0.3.4
  [3349acd9] + ProtoBuf v0.11.5
  [777ac1f9] + SimpleBufferStream v1.1.0
  [699a6c

In [2]:
import Pkg; Pkg.add("PyPlot")
import Pkg; Pkg.add("PyCall")

using PyPlot
using PyCall

# Define the locations
population_zones = Dict(
    "Zuihōden Temple" => (38.25076685689052, 140.8667686177859),
    "Sendai Castle" => (38.25209, 140.85592),
    "Sendai Station" => (38.26028322570647, 140.88239995371939),
    "Rakuten Seimei Park Miyagi" => (38.25695841274225, 140.90267074199306),
    "AER" => (38.26243, 140.88104),
    "Osaki Hachiman Shrine" => (38.27251, 140.84503),
    "Sendai Umino-Mori Aquarium" => (38.280300194102374, 140.98200323180546),
    "Sendai Trust Tower" => (38.25655, 140.87626),
    "Yurtec Stadium Sendai" => (38.31933922945424, 140.88186037724955),
    "Sendai Izumi PREMIUM OUTLETS" => (38.34201389944614, 140.83717960908587),
    "Sendai Mediatheque" => (38.26536, 140.86568),
    "Mitsui Outlet Park Sendai Port" => (38.27518, 140.98983),
    "Tohoku University Katahira Campus" => (38.25383, 140.87407),
    "Tohoku Electric Power Company" => (38.26555, 140.87835)
)

shelters = Dict(
    "錦ケ丘小" => (38.25644, 140.76093),
    "上杉山通小" => (38.27135, 140.87361),
    "広瀬中" => (38.27389, 140.76242),
    "仙台高" => (38.2821, 140.83406),
    "仙台青陵中等教育" => (38.28862, 140.83386),
    "榴岡小" => (38.26151, 140.89091),
    "岩切小" => (38.29979, 140.94251),
    "新田小" => (38.27628, 140.92843),
    "宮城野中" => (38.26269, 140.90001),
    "七郷小" => (38.24103, 140.94222),
    "蒲町小" => (38.24245, 140.92912),
    "東長町小" => (38.21892, 140.8892),
    "富沢小" => (38.20972, 140.86297),
    "長町中" => (38.23077, 140.87384),
    "富沢中" => (38.21849, 140.86325),
    "南光台小" => (38.30032, 140.9006),
    "仙台商業高" => (38.32415, 140.87201)
)

# Create a plot
fig, ax = subplots()

# Plot population zones in red
scatter([coord[2] for coord in values(population_zones)], [coord[1] for coord in values(population_zones)], c="red", label="Population Zones")

# Plot shelters in blue
scatter([coord[2] for coord in values(shelters)], [coord[1] for coord in values(shelters)], c="blue", label="Shelters")

# Connect the points with lines
for (zone_name, zone_coords) in population_zones
    for (shelter_name, shelter_coords) in shelters
        plot([zone_coords[2], shelter_coords[2]], [zone_coords[1], shelter_coords[1]], c="gray", linestyle="--")
    end
end

# Customize the plot
ax.set_xlabel("Longitude")
ax.set_ylabel("Latitude")
ax.set_title("Population Zones and Shelters")
ax.legend()

# Show the plot
show()

   Resolving package versions...


LoadError: Error when installing package PyCall:
IOError: mkdir("/Users/jakechon/.julia/packages/PyCall/twYvK"; mode=0o777): permission denied (EACCES)
Stacktrace:
 [1] uv_error
   @ ./libuv.jl:97 [inlined]
 [2] mkdir(path::String; mode::UInt16)
   @ Base.Filesystem ./file.jl:183
 [3] mkpath(path::String; mode::UInt16)
   @ Base.Filesystem ./file.jl:234
 [4] mkpath
   @ ./file.jl:229 [inlined]
 [5] install_archive(urls::Vector{Pair{String, Bool}}, hash::Base.SHA1, version_path::String; io::IJulia.IJuliaStdio{Base.PipeEndpoint})
   @ Pkg.Operations /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/stdlib/v1.7/Pkg/src/Operations.jl:521
 [6] macro expansion
   @ /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/stdlib/v1.7/Pkg/src/Operations.jl:698 [inlined]
 [7] (::Pkg.Operations.var"#45#50"{Bool, Pkg.Types.Context, Channel{Any}, Channel{NamedTuple{(:pkg, :urls, :path), Tuple{Pkg.Types.PackageEntry, Set{String}, String}}}})()
   @ Pkg.Operations ./task.jl:42

In [2]:
#NEW
#FOR IMMEDIATE SHELTER
#Insert data below
#Used 1 for  ̃

#GENERAL 
#Sets and Indices
I = [20,30,40] #Set of sources of evacuation, i ∈ I

J = 1:20 #Set of potential shelter locations, j ∈ J

l = #Set of different capacity levels of the potential shelters, l ∈ L

#Model Parameters/Data
pop = [1000,2000,3000] #Evacuee population at sources i ∈ I

op_cost = #Operations cost for each person in a shelter at shelter j ∈ J with level l ∈ L ($/person)

distance= #Distance between source i ∈ I and shelter location j ∈ J

t_avgcost = #Average transportation cost to move one person by unit distance ($/person-km)

t_cost = #(:= Distance*T-AvgCost) Transportation cost to move one person from source i to shelter j

cap = #Capacity of the shelter at location j ∈ J with level l ∈ L, and qj0 := 0 ∀j,l

maxd = #Maximum distance

@constraint(model,maxd)

#Decision Variables

x = #Number of people evacuating from source i ∈ I going to shelter j ∈ J

yj = #1, if a shelter of level l ∈ L opens at j ∈ J,0 otherwise

uj = #1, if source i ∈ I is allocated to shelter j ∈ J, 0 otherwise

current_cap = #number of people at shelter j using Lth level

LoadError: LoadError: UndefVarError: @constraint not defined
in expression starting at In[2]:29

In [2]:
using Distances

function distance(lat, lng, lat0, lng0)
    deglen = 110.25
    x = lat - lat0
    y = (lng - lng0) * cos(lat0)
    return deglen * sqrt(x*x + y*y)
end

#Data
locations_dict = Dict("Zuihōden Temple" => (38.25076685689052, 140.8667686177859),
"Sendai Castle" => (38.25209, 140.85592),
"Sendai Station" => (38.26028322570647, 140.88239995371939),
"Rakuten Seimei Park Miyagi" => (38.25695841274225, 140.90267074199306),
"AER" => (38.26243, 140.88104),
"Osaki Hachiman Shrine" => (38.27251, 140.84503),
"Sendai Umino-Mori Aquarium" => (38.280300194102374, 140.98200323180546),
"Sendai Trust Tower" => (38.25655, 140.87626),
"Yurtec Stadium Sendai" => (38.31933922945424, 140.88186037724955),
"Sendai Izumi PREMIUM OUTLETS" => (38.34201389944614, 140.83717960908587),
"Sendai Mediatheque" => (38.26536, 140.86568),
"Mitsui Outlet Park Sendai Port" => (38.27518, 140.98983),
"Tohoku University Katahira Campus" => (38.25383, 140.87407),
"Tohoku Electric Power Company" => (38.26555, 140.87835),
"Dummy-location Sendai Kousei Hospital" => (38.26979, 140.85759),
"Dummy-location Miyagi Children's Hospital" => (38.27483, 140.78239),
"Dummy-location JCHO Sendai Hospital" => (38.34563, 140.83174))

shelters_dict = Dict("錦ケ丘小" => (38.25644, 140.76093),
"上杉山通小" => (38.27135, 140.87361),
"広瀬中" => (38.27389, 140.76242),
"仙台高" => (38.2821, 140.83406),
"仙台青陵中等教育" => (38.28862, 140.83386),
"榴岡小" => (38.26151, 140.89091),
"岩切小" => (38.29979, 140.94251),
"新田小" => (38.27628, 140.92843),
"宮城野中" => (38.26269, 140.90001),
"七郷小" => (38.24103, 140.94222),
"蒲町小" => (38.24245, 140.92912),
"東長町小" => (38.21892, 140.8892),
"富沢小" => (38.20972, 140.86297),
"長町中" => (38.23077, 140.87384),
"富沢中" => (38.21849, 140.86325),
"南光台小" => (38.30032, 140.9006),
"仙台商業高" => (38.32415, 140.87201),
"Dummy-shelter Sendai Kousei Hospital" => (38.26979, 140.85759),
"Dummy-shelter Miyagi Children's Hospital" => (38.27483, 140.78239),
"Dummy-shelter JCHO Sendai Hospital" => (38.34563, 140.83174))

#Calculate
distances_dict = Dict{String, Dict{String, Float64}}()

for (loc_name, loc_coords) in locations_dict
    distances_dict[loc_name] = Dict{String, Float64}()
    
    for (sh_name, sh_coords) in shelters_dict
        distances_dict[loc_name][sh_name] = distance(loc_coords[1], loc_coords[2], sh_coords[1], sh_coords[2])
    end
end

#Print
for (loc_name, distances) in distances_dict
    println("Distances from $loc_name:")
    
    for (sh_name, distance) in distances
        println("  $sh_name: $distance km")
    end
    
    println()
end

Distances from Sendai Trust Tower:
  広瀬中: 10.706165212457629 km
  仙台青陵中等教育: 5.253488264906729 km
  Dummy-shelter Miyagi Children's Hospital: 8.911772985170613 km
  蒲町小: 5.225136877733076 km
  長町中: 2.851534005846954 km
  七郷小: 6.460850045301224 km
  新田小: 5.288125807409881 km
  仙台高: 4.797987507462382 km
  Dummy-shelter Sendai Kousei Hospital: 2.2652214753395485 km
  南光台小: 5.308855294814546 km
  錦ケ丘小: 10.790975241507619 km
  上杉山通小: 1.6500828140074453 km
  宮城野中: 2.3146910899436053 km
  仙台商業高: 7.462580016934648 km
  岩切小: 7.683293427436732 km
  榴岡小: 1.4717555808435177 km
  Dummy-shelter JCHO Sendai Hospital: 10.573659098858842 km
  東長町小: 4.329538796042483 km
  富沢小: 5.318907136556792 km
  富沢中: 4.376974755719223 km

Distances from Dummy-location Sendai Kousei Hospital:
  広瀬中: 8.818077886005584 km
  仙台青陵中等教育: 3.0064777198972616 km
  Dummy-shelter Miyagi Children's Hospital: 6.976499344402523 km
  蒲町小: 7.39286966238646 km
  長町中: 4.570731145965316 km
  七郷小: 8.59950690068839 km
  新田小: 6.58395553648

  仙台青陵中等教育: 4.9557523588319015 km
  Dummy-shelter Miyagi Children's Hospital: 0.0 km
  蒲町小: 14.300062451474009 km
  長町中: 9.956112224779774 km
  七郷小: 15.549621697263484 km
  新田小: 13.49369009359553 km
  仙台高: 4.822728661301053 km
  Dummy-shelter Sendai Kousei Hospital: 6.9990888204800905 km
  南光台小: 11.108759421209436 km
  錦ケ丘小: 2.8535093499409476 km
  上杉山通小: 8.463535217857816 km
  宮城野中: 11.04355874205729 km
  仙台商業高: 9.683401335677523 km
  岩切小: 14.8207900269879 km
  榴岡小: 10.227548688802516 km
  Dummy-shelter JCHO Sendai Hospital: 8.932461776785027 km
  東長町小: 11.935334710209029 km
  富沢小: 10.564261461145662 km
  富沢中: 9.923596607005324 km

Distances from Yurtec Stadium Sendai:
  広瀬中: 12.135147151546489 km
  仙台青陵中等教育: 5.551576361430548 km
  Dummy-shelter Miyagi Children's Hospital: 10.425839294080632 km
  蒲町小: 9.578721284848323 km
  長町中: 9.794458969827422 km
  七郷小: 10.346126478852183 km
  新田小: 6.406953211520052 km
  仙台高: 6.017621606982947 km
  Dummy-shelter Sendai Kousei Hospital: 5.9087015739

In [3]:
using Distances

function distance(lat, lng, lat0, lng0)
    deglen = 110.25
    x = lat - lat0
    y = (lng - lng0) * cos(lat0)
    return deglen * sqrt(x*x + y*y)
end

#Data
locations_dict = Dict("Zuihōden Temple" => (38.25076685689052, 140.8667686177859),
"Sendai Castle" => (38.25209, 140.85592),
"Sendai Station" => (38.26028322570647, 140.88239995371939),
"Rakuten Seimei Park Miyagi" => (38.25695841274225, 140.90267074199306),
"AER" => (38.26243, 140.88104),
"Osaki Hachiman Shrine" => (38.27251, 140.84503),
"Sendai Umino-Mori Aquarium" => (38.280300194102374, 140.98200323180546),
"Sendai Trust Tower" => (38.25655, 140.87626),
"Yurtec Stadium Sendai" => (38.31933922945424, 140.88186037724955),
"Sendai Izumi PREMIUM OUTLETS" => (38.34201389944614, 140.83717960908587),
"Sendai Mediatheque" => (38.26536, 140.86568),
"Mitsui Outlet Park Sendai Port" => (38.27518, 140.98983),
"Tohoku University Katahira Campus" => (38.25383, 140.87407),
"Tohoku Electric Power Company" => (38.26555, 140.87835),
"Dummy-location Sendai Kousei Hospital" => (38.26979, 140.85759),
"Dummy-location Miyagi Children's Hospital" => (38.27483, 140.78239),
"Dummy-location JCHO Sendai Hospital" => (38.34563, 140.83174))

shelters_dict = Dict("錦ケ丘小" => (38.25644, 140.76093),
"上杉山通小" => (38.27135, 140.87361),
"広瀬中" => (38.27389, 140.76242),
"仙台高" => (38.2821, 140.83406),
"仙台青陵中等教育" => (38.28862, 140.83386),
"榴岡小" => (38.26151, 140.89091),
"岩切小" => (38.29979, 140.94251),
"新田小" => (38.27628, 140.92843),
"宮城野中" => (38.26269, 140.90001),
"七郷小" => (38.24103, 140.94222),
"蒲町小" => (38.24245, 140.92912),
"東長町小" => (38.21892, 140.8892),
"富沢小" => (38.20972, 140.86297),
"長町中" => (38.23077, 140.87384),
"富沢中" => (38.21849, 140.86325),
"南光台小" => (38.30032, 140.9006),
"仙台商業高" => (38.32415, 140.87201),
"Dummy-shelter Sendai Kousei Hospital" => (38.26979, 140.85759),
"Dummy-shelter Miyagi Children's Hospital" => (38.27483, 140.78239),
"Dummy-shelter JCHO Sendai Hospital" => (38.34563, 140.83174))

# Calculate and store distances in a nested dictionary
distances_dict = Dict{String, Dict{String, Float64}}()

for (sh_name, sh_coords) in shelters_dict
    distances_dict[sh_name] = Dict{String, Float64}()
    
    for (loc_name, loc_coords) in locations_dict
        distances_dict[sh_name][loc_name] = distance(sh_coords[1], sh_coords[2], loc_coords[1], loc_coords[2])
    end
end

# Print the calculated distances grouped by shelter
for (sh_name, distances) in distances_dict
    println("Distances from $sh_name:")
    
    for (loc_name, distance) in distances
        println("  $loc_name: $distance km")
    end
    
    println()
end


Distances from 広瀬中:
  Sendai Trust Tower: 10.821035030067623 km
  Dummy-location Sendai Kousei Hospital: 8.841360571167662 km
  Zuihōden Temple: 10.124074494398766 km
  Osaki Hachiman Shrine: 7.652592675104762 km
  Tohoku University Katahira Campus: 10.694775592182788 km
  AER: 11.129119861789684 km
  Tohoku Electric Power Company: 10.824197134354714 km
  Sendai Izumi PREMIUM OUTLETS: 9.996361328556832 km
  Sendai Mediatheque: 9.653448988360147 km
  Sendai Station: 11.299101976186408 km
  Sendai Umino-Mori Aquarium: 20.24652508318375 km
  Mitsui Outlet Park Sendai Port: 21.02609423350555 km
  Dummy-location Miyagi Children's Hospital: 1.8496885628197133 km
  Yurtec Stadium Sendai: 11.829336187846215 km
  Sendai Castle: 9.095359074820491 km
  Dummy-location JCHO Sendai Hospital: 9.988470856502703 km
  Rakuten Seimei Park Miyagi: 13.250608449849672 km

Distances from 仙台青陵中等教育:
  Sendai Trust Tower: 5.313925867081004 km
  Dummy-location Sendai Kousei Hospital: 3.0260656339292873 km
  Zuih

  Dummy-location Miyagi Children's Hospital: 8.444551493927046 km
  Yurtec Stadium Sendai: 5.3423378940876205 km
  Sendai Castle: 2.6950591990390103 km
  Dummy-location JCHO Sendai Hospital: 8.980076275560274 km
  Rakuten Seimei Park Miyagi: 3.1474185738270393 km

Distances from 宮城野中:
  Sendai Trust Tower: 2.322865664732745 km
  Dummy-location Sendai Kousei Hospital: 4.012782849290165 km
  Zuihōden Temple: 3.3867292200828576 km
  Osaki Hachiman Shrine: 5.205898384620388 km
  Tohoku University Katahira Campus: 2.6199491468160714 km
  AER: 1.7685216311417211 km
  Tohoku Electric Power Company: 2.039570415084016 km
  Sendai Izumi PREMIUM OUTLETS: 10.354730253562403 km
  Sendai Mediatheque: 3.2076723356553147 km
  Sendai Station: 1.6650225860333887 km
  Sendai Umino-Mori Aquarium: 7.80099082590512 km
  Mitsui Outlet Park Sendai Port: 8.417867316332858 km
  Dummy-location Miyagi Children's Hospital: 10.959284097655543 km
  Yurtec Stadium Sendai: 6.454348125018127 km
  Sendai Castle: 4.29838

In [18]:
#no transshipment
using JuMP
import DelimitedFiles
import HiGHS

open(joinpath(@__DIR__, "transp.txt"), "w") do io
    print(
        io,
        """
             . 広瀬中   仙台青   蒲町小  長町中  七郷小      新田小  仙台高   南光台   錦ケ丘   上杉山   宮城野   仙台商   岩切小    榴岡小  東長町    富沢小  富沢中  EVACUEES
          TTWR 10.71	5.25	5.23	2.85	6.46	5.29	4.80	5.31	10.79	1.65	2.31	7.46	7.68	1.47	4.33	5.32	4.38  1178
          ZUHO 9.99 	5.15	5.96	2.30	7.21	6.35	4.58	6.27	9.92	2.36	3.37	8.10	8.76	2.54	4.12	4.54	3.57  60
          OSAK 7.65 	2.05	8.60	5.35	9.81	7.72	1.46	5.91	8.07	2.65	5.24	6.18	9.36	4.45	7.26	7.13	6.21  100000
          TDAI 10.57	5.32	5.34	2.54	6.59	5.60	4.82	5.66	10.59	1.93	2.61	7.75	8.03	1.78	4.11	4.98	4.03  5000
          AERT 11.05	5.20	5.04	3.56	6.24	4.64	4.84	4.54	11.26	1.20	1.77	6.85	6.94	0.93	4.86	6.07	5.13  773
          TEPC 10.77	4.81	5.43	3.86	6.61	4.78	4.47	4.33	11.03	0.78	2.04	6.49	6.95	1.25	5.24	6.33	5.39  12500
          SIPO 10.21	5.89	13.99	12.75	14.91	11.12	6.61	7.37	11.83	8.49	10.52	3.68	10.65	10.19	14.46	14.79	13.85 1653
          MEDI 9.60 	3.88	6.50	3.89	7.71	5.92	3.45	4.99	9.85	0.99	3.21	6.51	7.95	2.39	5.59	6.14	5.17  346
          SSTA 11.20	5.44	4.83	3.35	6.04	4.60	5.06	4.71	11.37	1.47	1.66	7.10	6.99	0.81	4.61	5.88	4.96  75608
          AQUA 20.33	13.61	6.51	11.64	5.73	4.97	13.62	7.72	20.85	10.09	7.88	10.96	4.19	8.75	11.16	13.84	13.25 913
          MOPS 21.04	14.37	6.77	12.06	5.86	5.67	14.36	8.57	21.52	10.78	8.48	11.84	5.09	9.35	11.45	14.18	13.63 10000
          YURT 12.14	5.55	9.58	9.79	10.35	6.41	6.02	2.70	13.27	5.35	6.47	1.03	5.92	6.43	11.09	12.22	11.26 11467
          CAST 8.98 	4.51	6.99	2.90	8.24	7.21	3.87	6.69	8.90	2.68	4.27	8.07	9.47	3.42	4.85	4.72	3.77  87
          RAKU 13.11	7.21	2.96	3.98	4.13	3.19	6.90	4.78	13.26	3.13	0.68	7.90	5.95	1.21	4.39	6.46	5.68  27000
        VACANCIES 15232 14955 12950 16378 12195 13261 21261 10464 14474 14445 14733 24328 16014 13779 12595 15111 18578 0
        """,
    )
    return
end

function read_data(filename::String)
    data = DelimitedFiles.readdlm(filename)
    rows, columns = data[2:end, 1], data[1, 2:end]
    return Containers.DenseAxisArray(data[2:end, 2:end], rows, columns)
end

data = read_data(joinpath(@__DIR__, "transp.txt"))
print(data)

function solve_transportation_problem(data::Containers.DenseAxisArray)
    # Get the set of supplies and demands
    O, D = axes(data)
    # Drop the EVACUEES and VACANCIES nodes from our sets
    O, D = setdiff(O, ["VACANCIES"]), setdiff(D, ["EVACUEES"])
    model = Model(HiGHS.Optimizer)
    set_silent(model)
    @variable(model, x[o in O, d in D] >= 0)
    # Remove arcs with "." cost by fixing them to 0.0.
    for o in O, d in D
        if data[o, d] == "."
            fix(x[o, d], 0.0; force = true)
        end
    end
    @objective(
        model,
        Min,
        sum(data[o, d] * x[o, d] for o in O, d in D if data[o, d] != "."),
    )
    @constraint(model, [o in O], sum(x[o, :]) == data[o, "EVACUEES"])
    @constraint(model, [d in D], sum(x[:, d]) <= data["VACANCIES", d])
    optimize!(model)
    # Pretty print the solution in the format of the input
    print("    ", join(lpad.(D, 7, ' ')))
    for o in O
        print("\n", o)
        for d in D
            if isapprox(value(x[o, d]), 0.0; atol = 1e-6)
                print("      .")
            else
                print(" ", lpad(value(x[o, d]), 6, ' '))
            end
        end
    end
    return
end

print("\nSolution:\n")
solve_transportation_problem(data)

#########################################

function calculate_numerical_derivative(data::JuMP.Containers.DenseAxisArray{Any, 2}, epsilon::Float64 = 1)
    O, D = axes(data)
    O, D = setdiff(O, ["VACANCIES"]), setdiff(D, ["EVACUEES"])
    num_rows, num_cols = length(O), length(D)
    derivative_matrix = zeros(Float64, num_rows, num_cols)

    for (i, o) in enumerate(O), (j, d) in enumerate(D)
        original_cost = data[o, d]

        if original_cost isa Number
            # Increase the cost slightly and re-solve
            data[o, d] = original_cost + epsilon
            increased_cost_model = Model(HiGHS.Optimizer)
            set_silent(increased_cost_model)
            @variable(increased_cost_model, x[ii in O, jj in D] >= 0)

            for ii in O, jj in D
                if data[ii, jj] == "."
                    fix(x[ii, jj], 0.0; force = true)
                end
            end

            @objective(
                increased_cost_model,
                Min,
                sum(data[ii, jj] * x[ii, jj] for ii in O, jj in D if data[ii, jj] isa Number),
            )

            @constraint(increased_cost_model, [ii in O], sum(x[ii, :]) == data[ii, "EVACUEES"])
            @constraint(increased_cost_model, [jj in D], sum(x[:, jj]) <= data["VACANCIES", jj])

            optimize!(increased_cost_model)
            increased_cost_objective = objective_value(increased_cost_model)

            # Decrease the cost slightly and re-solve
            data[o, d] = original_cost - epsilon
            decreased_cost_model = Model(HiGHS.Optimizer)
            set_silent(decreased_cost_model)
            @variable(decreased_cost_model, x[ii in O, jj in D] >= 0)

            for ii in O, jj in D
                if data[ii, jj] == "."
                    fix(x[ii, jj], 0.0; force = true)
                end
            end

            @objective(
                decreased_cost_model,
                Min,
                sum(data[ii, jj] * x[ii, jj] for ii in O, jj in D if data[ii, jj] isa Number),
            )

            @constraint(decreased_cost_model, [ii in O], sum(x[ii, :]) == data[ii, "EVACUEES"])
            @constraint(decreased_cost_model, [jj in D], sum(x[:, jj]) <= data["VACANCIES", jj])

            optimize!(decreased_cost_model)
            decreased_cost_objective = objective_value(decreased_cost_model)

            # Calculate the numerical derivative
            derivative = (increased_cost_objective - decreased_cost_objective) / (2 * epsilon)
            derivative_matrix[i, j] = derivative

            # Reset the original cost
            data[o, d] = original_cost
        end
    end

    return derivative_matrix
end

# Calculate and print the numerical derivative matrix
epsilon = 1.0
derivative_matrix = calculate_numerical_derivative(data, epsilon)
println("\nNumerical Derivative Matrix:")
for row in eachrow(derivative_matrix)
    println(row)
end

2-dimensional DenseAxisArray{Any,2,...} with index sets:
    Dimension 1, Any["TTWR", "ZUHO", "OSAK", "TDAI", "AERT", "TEPC", "SIPO", "MEDI", "SSTA", "AQUA", "MOPS", "YURT", "CAST", "RAKU", "VACANCIES"]
    Dimension 2, Any["広瀬中", "仙台青", "蒲町小", "長町中", "七郷小", "新田小", "仙台高", "南光台", "錦ケ丘", "上杉山", "宮城野", "仙台商", "岩切小", "榴岡小", "東長町", "富沢小", "富沢中", "EVACUEES"]
And data, a 15×18 Matrix{Any}:
    10.71      5.25      5.23      2.85      6.46      5.29      4.8       5.31     10.79      1.65      2.31      7.46      7.68      1.47      4.33      5.32      4.38    1178
     9.99      5.15      5.96      2.3       7.21      6.35      4.58      6.27      9.92      2.36      3.37      8.1       8.76      2.54      4.12      4.54      3.57      60
     7.65      2.05      8.6       5.35      9.81      7.72      1.46      5.91      8.07      2.65      5.24      6.18      9.36      4.45      7.26      7.13      6.21  100000
    10.57      5.32      5.34      2.54      6.59      5.6       4.82      5.66 

In [ ]:
##OLD

using JuMP
using HiGHS
model = Model(HiGHS.Optimizer)

#All Variables
@variable(model, x[I,J]>= 0) #evacuee population
@variable(model, yjl[L,J],Bin) #shelter opening is binary
@variable(model, uij[I,J],Bin) #source allocation is binary

#Constraints
for i in I 
    for j in J
        if distance[i,j]>maxd 
            fix(u[i,j],0) #if the distance is greater than the max, source is not allocated to shelter j

@constraint(model,[j in J], sum(x[i,j] for i in I)<=sum(t[j,l] for l in L)) #line 6 the all evacuees at shelter can be accommodated
@constraint(model,[p in P], sum(S[k,p] * Z1[k] for k in K)>=W[p]*sum(pop[i] for i in I)) #line 9
@constraint(model,[j in J], sum(cap[j,l] * y[j,l] for j in J, for l in L)>=sum(pop[i] for i in I))#line 10 makes sure population is over
@constraint(model,[i in I, j in J],x[i,j]<=M*u[i,j]) #line 11 pt1
@constraint(model,[i in I, j in J] u[i,j]<=sum(y[j,l] for l in L))#line 11 pt2

#new
@objective(model,Min,(sum(op_cost[j,l] for j in J)+(sum(t_cost[i,j] for i in I)))